# Implementation of algorithm from Diaz and van der Laan

In [12]:
rm(list = ls())
set.seed(429153)
n_obs <- 1000
n_w <- 3

In [23]:
# simulate simple data for tmle-shift sketch
W <- replicate(n_w, rnorm(n_obs))
A <- rowSums(cos(exp(W)) + W)
Y <- sin(A)
O <- as.data.frame(cbind(W,A,Y))
colnames(O) <- c(paste0("W", seq_len(n_w)), "A", "Y")
head(O)

W1,W2,W3,A,Y
0.1080803,-0.230222921,-0.4744050,1.3577145,0.9773838
0.5021332,-0.291323049,0.5347912,1.2619159,0.9526745
-1.8953431,-0.656984062,0.9888629,-0.6050819,-0.5688294
-1.2538256,-0.001461187,-0.5695259,0.5204272,0.4972508
0.3002404,-0.296505112,0.2389099,1.4940473,0.9970562
-0.8670006,-0.264386145,-1.3021535,0.1622390,0.1615282


# Anatomy of the shift-Tx package

The algorithm is based on @diaz2017stochastic.

## Starting Assumptions

1. Start with a simple additive shift -- i.e., $d(a,w) = a + \delta$ if $a <
    u(w) - \delta$ or $d(a,w) = a$ if $a \geq u(w) - \delta$.
2. The additive shift will have _support everywhere_ -- i.e., $a < u(w)$ is true
    everywhere.
3. The data structure that we know and love $O = (W,A,Y)$.

## Functions Needed

* estimate $g_n(W)$
* estimate $Q_n(A, W)$
* estimate auxiliary covariate $H_n(A_i, W_i)$
* fluctuation procedure
* 1-TMLE procedure
* EIF procedure

## Estimate $g_n(W)$

* _input_: W, a
* _output_: a 2-column matrix, with columns for $g_n(A_i - \delta \mid W_i)$ and
    $g_n(A_i \mid W_i)$
* in the inputs $a$ is the additive shift
* use the __fit_density__ function from Oleg's __condensier__ package, need to
    use __predict_prob__ function twice: once for $A_i - \delta$ and once for
    $A_i$

## Estimate $Q_n(A, W)$

* _input_: W, a
* _output_: a 2-column matrix, with columns for $\bar{Q}_n(A_i, W_i)$ and
    $\bar{Q}_n(A_i + \delta, W_i)$

## Estimate $H_n(A_i, W_i)$

* _input_: matrix output produced by $g_n(w)$
* _output_: vector (possibly shifted) of the form described in the eqn below
* $H(a,w) = I(a < u(w)) \frac{g_0(a - \delta \mid w)}{g_0(a \mid w)} + I(a
    \geq u(w) - \delta)$
* By our assumption (2) above -- that we have _support everywhere_ -- we reduce
    the above formulation
* That is, we assume that $I(a < u(w)) = 1$ and $I(a \geq u(w) - \delta) = 0$
* Thus the form of the covariate reduces simply to $H(a,w) = \frac{g_0(a -
    \delta \mid w)}{g_0(a \mid w)}$

## Fluctuation Procedure

* _input_: matrix output from $Q_n(a,w)$, vector output of $H_n$, vector Y
* _output_: model fit object produced from a call to `glm` or `SuperLearner`
* We have the fluctuation model: $logit \bar{Q}_{\epsilon, n}(a,w) =
    logit(\bar{Q}_n(a,w)) + \epsilon \cdot H_n(a,w)$
* Note that the first term on the RHS of the above equation is one of the
    columns generated as output by the function to estimate $Q_n(A,W)$
* this could be fit with R code like the following `glm(Y ~ -1 +
    offset(logitQn_AW + Hn_AW), family = "binomial")`, from which we may extract
    the coefficient, which is $\epsilon_n$ from the above

## 1-TMLE Procedure

* _input_: model fit object produced by the fluctuation procedure above, matrix
    produced by procedure to estimate $Q_n(A,W)$
* _output_: numeric scalar for the mean of $\bar{Q}^*_n$
* note that we have $\psi_n = \frac{1}{n} \sum_{i=1}^n \bar{Q}_n^*(d(A_i, W_i),
    W_i)$
* we obtain $\bar{Q}_n^*$ by calling the appropriate method of predict on the
    shifted data -- i.e., `predict(fit, newdata = data.frame(Qn_dAW), type =
    "response"` (note that use of 'response' performs the `expit()` transform).
* compute the $\psi_n$ as the mean of the vector produced by calling `predict`
    on the fit object, as described above

## EIF Procedure

* _input_: matrix produced by $Q^*$: a 2-column matrix, with columns for
    $\bar{Q}_n(A_i, W_i)$ and $\bar{Q}_n(A_i + \delta, W_i)$
* _output_: scalar, the variance of the efficient influence function
* note that we have the _efficient influence function_ (EIF): $D(P)(o) =
    H(a,w)(y - \bar{Q}(a,w)) + \bar{Q}(d(a,w)) - \psi(P)$
* to compute the EIF from the above, we may set up a function like the following
    `eif <- function(Y, H, Qn_AW, Qn_dAW, Psi)`, which can then compute $\psi$
    by calling 1-TMLE (alternatively, the mean of the vector `Qn_dAW`) and then
    using the formula above
* compute $\sigma^2_n = \frac{1}{n}(EIF^2)$, that is simply call mean on the
    vector produced by the above